In [1]:
!nvidia-smi

Fri Nov 19 15:29:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    77W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U transformers datasets -q

In [3]:
import pandas as pd
from transformers import AutoTokenizer

path='../input/jvc20k/jvc-20k.csv'
dataset = pd.read_csv(path)

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

def get_training_corpus(dataset):
    for i in range(0, len(dataset["data"]), 1000):
        yield dataset["data"][i : i + 1000]

training_corpus = get_training_corpus(dataset)

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # Add PADDING token

tokenizer.save_pretrained("pretrained/jvc-tokenizer")

('pretrained/jvc-tokenizer/tokenizer_config.json',
 'pretrained/jvc-tokenizer/special_tokens_map.json',
 'pretrained/jvc-tokenizer/vocab.json',
 'pretrained/jvc-tokenizer/merges.txt',
 'pretrained/jvc-tokenizer/added_tokens.json',
 'pretrained/jvc-tokenizer/tokenizer.json')

In [ ]:
from datasets import load_dataset
from transformers import (AutoTokenizer, 
                         GPT2LMHeadModel, 
                         TrainingArguments, 
                         Trainer, 
                         DataCollatorForLanguageModeling)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("pretrained/jvc-tokenizer/")

def tokenize_function(examples):
    return tokenizer(examples["data"], truncation=True, max_length=256)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False) # Dynamic Padding

# Dataset
dataset = load_dataset('csv', data_files='../input/jvc20k/jvc-20k.csv') 
tokenized_dataset = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_dataset["train"]

# Model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Training
training_args = TrainingArguments(
    output_dir="pretrained/gpt2-jvc", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=30, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    save_strategy="no",
    remove_unused_columns=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()
trainer.save_model()

Using custom data configuration default-829a58f40360df3e
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-829a58f40360df3e/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-829a58f40360df3e/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-bbae37a0187a6827.arrow
The following columns in the training set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: data, content, id, title, author.
***** Running training *****
  Num examples = 20910
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 130700


Step,Training Loss


In [ ]:
from transformers import pipeline
from tqdm import tqdm

n_each = 100

shitpost = pipeline(
    task='text-generation',
    model='pretrained/gpt2-jvc',
    tokenizer="pretrained/gpt2-jvc",
    max_length=100,
    num_return_sequences=n_each
);

seq_init = [
    'AYAAAAA ',
    'Les golems ',
    'Alain Soral ',
    'Pourquoi ',
    'Comment ',
    '[BORDEL] ',
    '[ALERTE] ',
    '[PHOTO] ',
    'Les ',
    'Ton patron: ',
    'La caissière: ',
    'Ces ',
    'Les "mecs" qui ',
    'Ton excuse ',
    'Des kheys ',
    'Est-ce que ',
]

topic_list = []

for init in tqdm(seq_init):
    current = shitpost(init)
    topic_list += list(map(lambda x: x['generated_text'],current))
        
df = pd.DataFrame(topic_list)
df.to_csv(f'topic-{n_each*len(seq_init)}.csv')

loading configuration file ./pretrained/gpt2-jvc/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 50257
}

loading

In [ ]:
shitpost('Les golems')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Les golems vous trouvez un forum\nnan des poirier?\nAllez hop hop hop hop hop 2 ans\nLes golems vont sont effet la victoire ou tout faire, peut bien plus de 2 ans, je me montre et l'intérieur pour la mec de notre pouvais tout quelqu'un qui est mal à peut bien pas, c'est en chercher ou presque chose à plus de 2 ans à la mec vous trouvez pas?\nEn gros ça j'ai pas d'être pas moi vous vous pensez"},
 {'generated_text': "Les golems face à livreur vous jouez le kheys.\n\nEn France couille face et de livreur vous jouez le kheys ne pas faire un golems face à livreur vous jouez à votre ville contre un pays? Ca obèse? Ça pour un pays de l'amour l'étranger pour le pays? Ca transgenres? Ça quand les pays de l'époque.\nJe suis pas les kheys ici est un pays d'une face et tes vous jouez les kheys.\nD'ailleurs le pays? Ca obèse"},
 {'generated_text': "Les golems : Tiktok, c'est quoi pas, c'est pas quoi à bon-exercice\nAlors les kheys, et maintenant?\nPerso j'ai plus de la populati